# Optimization

Optimization for One-Variable Functions
- Analytical Method
- Golden Section Method
- Method with Derivatives 

Optimization for Multivariable Functions
- Gradient Descent Method
- Simplex Method (Nelder-Mead Algorithm)



## Analytical Optimization Method

To find the critical points of $ f(x) $, where the function achieves local minima or maxima:

1. Find the Derivative:
   - Compute the first derivative of the function:
     $
     f'(x)
     $

2. Solve for Critical Points:
   - Solve $ f'(x) = 0 $ to find the critical points $ x_c $.


In [54]:
def derivative(f,x,h=0.01):
    return (f(x+h)-f(x-h))/(2*h)

def bisection(f,a,b):
    if (f(a)*f(b)>0): return 
    while abs(a-b)>1e-6:
        c=(a+b)/2
        if (f(a)*f(c)<0):
            b=c
        else:
            a=c
    return c

def analytical_extrema(f,a,b, N=100):
    dx=(b-a)/(N-1)
    def f_deriv(x): return derivative(f, x)
    b=a + dx
    for i in range(N):
        extrema=bisection(f_deriv,a,b)
        if extrema!=None:
            return extrema
        a=a + i * dx
        b=a + (i + 1) * dx

## Golden Section Method

Let $ f(x)$ be a continuous function with an extreme on the interval $[a, b]$.

### Minimum

We choose two points $ c $ and $ d $ on the interval $[a, b]$, such that $ c - a = b - d $. <p></p>
We compute $ f(c) $ and $ f(d) $, if $ f(c) \leq f(d) $, we move $ b $ to $ d $ ($ b = d $);  Otherwise, we move $ a $ to $ c $ ($ a = c $).



Let us introduce shrinking ratio $$ r = \frac{d - a}{b - a} $$ When we shrink the interval, the new ratio is:  
$$
r = \frac{c - a}{d - a}
$$
where $ c $ has the role of $ d $ on the interval $[a, d]$. 

**We now require that this ratio be constant through all interval shrinking.**

From the first definition, we find:
$$
d = a + r(b - a)
$$
Then we use the condition $ b - d = c - a $ to express $ c $ with $ r $:
$$
c = a + (1 - r)(b - a)
$$

By our condition:
$$
\frac{d-a}{b-a} = \frac{c-a}{d-a}
$$

Plugging $ d $ and $ c $:
$$
\frac{r(b - a)}{b - a} = \frac{(1 - r)(b - a)}{r(b - a)}
$$

Solving this gives:
$$
r^2 + r - 1 = 0
$$
and
$$
r = \frac{-1 + \sqrt{5}}{2}
$$
This is the golden ratio.

### Algorithm for Minimum

1. Compute $ c $ and $ d $.
2. Check if $ f(c) \leq f(d) $:
   - If true: $ b = d, a = a, d = c, c = a + (1 - r)(b - a) $.
   - Else: $ a = c, b = b, c = d, d = a + r(b - a) $.
3. Continue shrinking until $ |b - a| < \epsilon $.

---

### Algorithm for Maximum

1. Compute $ c $ and $ d $.
2. Check if $ f(c) \geq f(d) $:
   - If true: $ b = d, a = a, d = c, c = a + (1 - r)(b - a)$.
   - Else: $ a = c, b = b, c = d, d = a + r(b - a) $.
3. Continue shrinking until $ |b - a| < \epsilon $.

This method is slow, because we compute everythin at each step. 

In [57]:
def golden_section(f,a,b,typ=0, ep=1e-6):
    from math import sqrt
    g = f if typ == 0 else lambda x: -f(x)
    r=(-1+sqrt(5))/2.0
    d=a+r*(b-a)
    c=a + (1-r)*(b-a)
    while abs(a-b)>ep:
        if (g(c)<=g(d)):
            b=d
            d=c
            c=a + (1-r)*(b-a)
        else:
            a=c
            c=d
            d=a+r*(b-a)
    return (a+b)/2
            
            
        

In [61]:
#Example
#y=x^2 on [-1,1]
def f(x):
    return x*x
min_golden=golden_section(f,-1,1,0)
print(min_golden)

min_anal=analytical_extrema(f,-1,1)
print(min_anal)

-2.0530310224463777e-07


NameError: name 'analytical_extrema' is not defined

## Method with Derivatives

Let $ f(x) $ be continuous and differentiable on $[a, b]$, with one maximum or minimum on that interval.


### Algorithm 
1. **Initial Selection**  
   Select a starting point $ x_0 $ and compute $ y_0 = f(x_0) $.
2. **Derivative Test**  
   Compute $ y'(x_0) $:  
   - If $ y'(x_0) < 0 $, then $ h < 0 $ (minimum lies to the left).  
   - If $ y'(x_0) > 0 $, then $ h > 0 $ (minimum lies to the right).  
3. **Choose Step Size $ h $)**  
   Select an initial step size $ h $.
4. **Compute Additional Points**  
   Compute:
   $$
   x_1 = x_0 + h, \quad x_2 = x_0 + 2h
   $$  
   and their corresponding values:
   $$
   y_1 = f(x_1), \quad y_2 = f(x_2).
   $$
5. **Check for Minimum**  
   For a minimum, the values should follow the order: $ y_0 > y_1 < y_2 $.  
   - If $ y_0 > y_1 > y_2 $: The step $ h $ is too small; update $h = 2h $.  
   - If $ y_0 < y_1 $: The step $ h $ is too large; update $ h = h/2$.  

6. **Construct Parabola**  
   Once the points are in the correct order, construct the parabola $ p(x) $ through $ (x_0, y_0), (x_1, y_1), (x_2, y_2) $:
   $$
   p(x) = \frac{(x - x_1)(x - x_2)}{2h^2} y_0 - \frac{(x - x_0)(x - x_2)}{h^2} y_1 + \frac{(x - x_0)(x - x_1)}{2h^2} y_2
   $$

   Acknowledge:
   $$
   x_1 - x_0 = h, \quad x_2 - x_1 = h, \quad x_2 - x_0 = 2h.
   $$
   Simplify:
   $$
   p(x) = \frac{(x - x_1)(x - x_2) y_0}{2h^2} - \frac{(x - x_0)(x - x_2) y_1}{h^2} + \frac{(x - x_0)(x - x_1) y_2}{2h^2}.
   $$
7. **Find Minimum of $ p(x) $**  
   Take the derivative $ p'(x) = 0 $ to find the minimum. Use $x=x_{min}$ at $x=x_0+h_{min}$:
   $$
   h_{\text{min}} = h \cdot \frac{3y_0 - 4y_1 + y_2}{2y_0 - 4y_1 + 2y_2}.
   $$

   The minimum of $ p(x) $ is at:
   $$
   x_{\text{min}} = x_0 + h_{\text{min}}.
  $$

   **Important**: This is the interpolated parabola's minimum, not the actual function's minimum.

8. **Iterate**  
   Update:
   $$
   x_0 = x_0 + h_{\text{min}}, \quad h = h_{\text{min}}.
   $$
   Repeat steps until $ |h_{\text{min}}| < \epsilon $.

 This method is computationally expensive but converges faster than the golden section method.



In [ ]:
def derivative_method(f,x0,h,ep=1e-6):
    df=lambda x: (f(x+0.01)-f(x-0.01))/(0.02)
    c=0
    while abs(h)>ep:
        if df(x0)>0: h=-abs(h)
        else: h=abs(h)
        y0=f(x0)
        y1=f(x0+h)
        y2=f(x0+2*h)
        if (y0>y1):
            if (y1<y2): #optimum loop
                h=h*(3*y0-4*y1+y2)/(2*y0-4*y1+2*y2)
                x0=x0+h
            else:
                h=2.0*h
        else:
            h=h/2.0
        c+=1
        print(h)
        if c>1000: break
                
    return x0



In [ ]:
#Example
#y=x^2 on [-1,1]
def f(x):
    return x*x*x+(x-1)*(x-1)

derivative_method(f,2,0.1)

0.548583646627351

In [4]:
import numpy as np
import matplotlib.pyplot as plt

x=np.linspace(-5,5,100)
plt.plot(x,f(x))


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\domin\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\domin\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\domin\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\domin\AppData\Local\Programs\Python\Python310\lib\site-packages\trait

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

## Gradient method

#### Notation
- Let the function be defined as:
  $$
  f(x_1, x_2, x_3, \dots, x_N) = f(\mathbf{x}),
  $$
  where $ \mathbf{x} = (x_1, x_2, \dots, x_N) $ is a vector of independent variables in $ N $-dimensional space.

- The gradient of $ f(\mathbf{x}) $ is denoted as:
  $$
  \nabla f(\mathbf{x}) = \left( \frac{\partial f(\mathbf{x})}{\partial x_1}, \frac{\partial f(\mathbf{x})}{\partial x_2}, \dots, \frac{\partial f(\mathbf{x})}{\partial x_N} \right).
  $$

This provides the direction of the steepest ascent of $f(\mathbf{x}) $.

---

#### Algorithm for Minimization
To find the minimum of $ f(\mathbf{x}) $ using the gradient method:

1. **Initialization**  
   Start with an initial guess $ \mathbf{x}_0 = (x_{1,0}, x_{2,0}, \dots, x_{N,0}) $.


2. **Compute Direction Vector**  
   Evaluate $\nabla f(\mathbf{x}_k) $, where $ \mathbf{x}_k $ is the current point.

   - Compute the direction vector $ \mathbf{s}_0 = -\nabla f(\mathbf{x}_k) $.
   - Normalize the direction vector:
    $$
     \mathbf{s}_0' = \frac{\mathbf{s}_0}{\|\mathbf{s}_0\|}.
     $$
     **Why normalize?**  
     Normalization ensures that the step size is independent of the magnitude of the gradient and focuses solely on direction.
3. **Find Optimal Step Size**  
   Find the optimal scalar $ \gamma $ (step size) by minimizing the one-variable function $ f(\mathbf{x}_k + \gamma \mathbf{s}_0') $ with respect to $ \gamma $. This determines how far to move along $ \mathbf{s}_0' $ in the current iteration.

4. **Update Step**  
   Update the point using:
   $$
   \mathbf{x}_{k+1} = \mathbf{x}_k + \gamma \mathbf{s}_0'.
   $$

5. **Convergence Check**  
   Repeat steps 2 through 4 until:
   $$
   \|\nabla f(\mathbf{x}_k)\| < \epsilon,
   $$
   where $ \epsilon $ is a small tolerance value.

---

#### Notes
- Normalization of the direction vector is particularly useful when performing line search for $ \gamma $ to ensure numerical stability.
- The method converges faster for well-behaved convex functions and may require additional techniques for non-convex functions to avoid getting trapped in local minima.

In [6]:
#Gradient descent
def gradient(P:list, f, df=None):
    grad=[]
    if df==None:
        def df(P,var,h=0.01):
            Pp=P.copy()
            Pm=P.copy()
            Pp[var]+=h
            Pm[var]-=h
            return (f(Pp)-f(Pm))/(2*h)
    for i in range(len(P)):
        grad.append(df(P,i))
    return grad

class vector():

    def __init__(self,x:list):
        self.data=x
        
    def norm(self):
        normal=0
        for xi in self.data:
            normal+=xi*xi
        return normal**0.5

    def normalization(self):
        normal=self.norm()
        for i in range(len(self.data)):
            self.data[i]=self.data[i]/normal

    def __call__(self):
        return self.data 

    def __rmul__(self,g):
        return vector([g*xi for xi in self.data])

    def __add__(self,second):
        if len(self.data) != len(second.data):
                raise ValueError("")
        return vector([x+y for x,y in zip(self.data,second.data)]) 
        
def simple_one_dimenional(f,a,b,*args):
    from math import sqrt
    r=(-1+sqrt(5))/2.0
    d=a+r*(b-a)
    c=a + (1-r)*(b-a)
    while abs(a-b)>1e-6:
        if (f(c,*args)<=f(d,*args)):
            b=d
            d=c
            c=a + (1-r)*(b-a)
        else:
            a=c
            c=d
            d=a+r*(b-a)
    return (a+b)/2

def gradient_descent(P0:list, f, df=None, steps=1000):
    P=vector(P0)
    for step in range(steps):
        s=gradient(P(),f,df)
        s=-1.0*vector(s)
        s_norm=s.norm()
        if (s_norm)<1e-4: break
        s.normalization()
        def line(g,P0,s,func):
            P=P0+(g*s)
            return func(P())
        g_min=simple_one_dimenional(line,0,10*s_norm,P,s,f)
        P=P+(g_min*s)
    return P()
        
        
        

In [8]:
def function(x:list):
    return (x[0]-x[1])/ ( x[0]*x[0] + x[1]*x[1] + 2 )
P=[3,2]
gradient_descent(P,function)

[-1.000015695134705, 0.9999949910837915]

#### Issue of All These Methods  

A key limitation of these optimization methods is their inability to escape local minima. Once the algorithm converges to a local minimum, it cannot differentiate whether it is a global minimum or just a smaller basin in the optimization landscape.  

##### Why This Happens
- **Gradient-Based Methods**: They rely on the local slope (gradient) to determine the next move. If the gradient leads to a local minimum, the method stops progressing.
- **Golden Section and Derivative-Based Methods**: These search for the minimum within a defined range and cannot explore outside of it. 

##### Implications  
In problems with multiple local minima, these methods may not find the globally optimal solution unless:
- The initial guess is close to the global minimum.
- Specialized strategies (like stochastic approaches) are incorporated to escape local minima.


## Simplex Method (Nelder-Mead Algorithm)

The Nelder-Mead algorithm is an optimization technique to minimize a function $ f(\mathbf{x}) $ of $ N $ independent variables without requiring derivatives. 

### Algorithm for Finding the Minimum

1. **Define the Function**  
   Let $ f(\mathbf{x}) = f(x_1, x_2, \dots, x_N) $, a function of $ N $ independent variables.

2. **Initialization**  
   Select $ N+1 $ starting points $ \mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_{N+1} $ in the space of the function.

3. **Evaluate the Function**  
   Compute $ f(\mathbf{x}_i) $ for all $ i = 1, 2, \dots, N+1 $.

4. **Sort the Points**  
   Sort the points based on the function values:
   $$
   f(\mathbf{x}_1) \leq f(\mathbf{x}_2) \leq \dots \leq f(\mathbf{x}_{N+1})
   $$
   Here, $ \mathbf{x}_1 $ is the best point, and $ \mathbf{x}_{N+1}$ is the worst point.

5. **Calculate the Centroid**  
   Compute the centroid $ \mathbf{x}_T $ of all points excluding the worst point $ \mathbf{x}_{N+1} $:
   $$
   \mathbf{x}_T = \left( \overline{x_1}, \overline{x_2}, \dots, \overline{x_N} \right)
   $$
   where the average for each coordinate is given by:
   $$
   \overline{x_i} = \frac{1}{N} \sum_{j=1}^{N} x_{i,j}
   $$
   Here:
   - Best Point: $ \mathbf{B} = \mathbf{x}_1 $
   - Worst Point: $ \mathbf{W} = \mathbf{x}_{N+1} $
   - Centroid: $ \mathbf{T} $
   - Good points: $\mathbf{G}$
6. **Reflection**  
   Reflect the worst point $ \mathbf{W} $ across the centroid $ \mathbf{T} $ to generate a new point $ \mathbf{x}_R $:
   $$
   \mathbf{x}_R = \mathbf{x}_T + \alpha (\mathbf{x}_T - \mathbf{W})
   $$
   where $ \alpha > 0 $ is the reflection coefficient (typically $ \alpha = 1 $).

   - Compute $ f(\mathbf{x}_R) $:
     - If $ f(\mathbf{x}_1) \leq f(\mathbf{x}_R) < f(\mathbf{x}_{N}) $, replace $ \mathbf{W} $ with $ \mathbf{x}_R $ and go back to Step 5.
     - If $ f(\mathbf{x}_R) < f(\mathbf{x}_1) $, proceed to **Expansion**.

7. **Expansion**  
   If the reflection improved the function significantly, perform an expansion to explore further in the same direction:
   $$
   \mathbf{x}_E = \mathbf{x}_T + \gamma (\mathbf{x}_R - \mathbf{x}_T)
   $$
   where $ \gamma > 1 $ is the expansion coefficient (typically $ \gamma = 2 $).

   - Compute $ f(\mathbf{x}_E) $:
     - If $ f(\mathbf{x}_E) < f(\mathbf{x}_R) $, replace $ \mathbf{W} $ with $ \mathbf{x}_E$.
     - Otherwise, replace $ \mathbf{W} $ with $ \mathbf{x}_R $.
   - Go back to Step 5.

8. **Contraction**  
   If reflection does not improve the function $( f(\mathbf{x}_R) \geq f(\mathbf{x}_N) )$ , perform a contraction to shrink the simplex towards the centroid:
   - **Outside Contraction** (if $ f(\mathbf{x}_1) \leq f(\mathbf{x}_R) < f(\mathbf{x}_{N+1}) $):
     $$
     \mathbf{x}_C = \mathbf{x}_T + \beta (\mathbf{x}_R - \mathbf{x}_T)
    $$
     where $ 0 < \beta < 1 $ is the contraction coefficient (typically $ \beta = 0.5 $).
   - **Inside Contraction** (if $f(\mathbf{x}_R) \geq f(\mathbf{x}_{N+1}) $):
     $$
     \mathbf{x}_C = \mathbf{x}_T + \beta (\mathbf{W} - \mathbf{x}_T)
     $$

   - Compute $ f(\mathbf{x}_C) $:
     - If $ f(\mathbf{x}_C) < f(\mathbf{x}_{N+1}) $, replace $ \mathbf{W} $ with $ \mathbf{x}_C $.
     - Otherwise, proceed to **Reduction**.

9. **Reduction**  
   If contraction fails, reduce the simplex by moving all points towards the best point $ \mathbf{B} $:
   $$
   \mathbf{x}_i = \mathbf{x}_1 + \delta (\mathbf{x}_i - \mathbf{x}_1)
   $$
   where $ 0 < \delta < 1 $ is the reduction coefficient (typically $ \delta = 0.5 $).

10. **Convergence Check**  
    Repeat the process until the simplex size or the function value difference between points is below a pre-specified tolerance:
    $$
    \| \mathbf{x}_i - \mathbf{x}_j \| < \epsilon \quad \text{and} \quad |f(\mathbf{x}_i) - f(\mathbf{x}_j)| < \epsilon
    $$
    for all $ i, j $.

### Notes
- The algorithm is particularly effective for non-differentiable functions and noisy data.
- The coefficients $ \alpha$, $\gamma $, $\beta $, and $ \delta $ control the behavior of the algorithm:
  - $ \alpha $: Reflection (exploration)
  - $ \gamma $: Expansion (aggressive exploration)
  - $ \beta $: Contraction (fine-tuning)
  - $ \delta $: Reduction (resetting)

This iterative process allows the Nelder-Mead method to converge to a local minimum of $ f(\mathbf{x}) $, provided it is continuous and bounded.

In [140]:
#Simplex
def function(x:list):
    return (x[0]-x[1])/ ( x[0]*x[0] + x[1]*x[1] + 2 )

class simplex():

    def __init__(self, function,points : list[list]):
        self.f=function
        if len(points)-1 !=len(points[0]):
             raise ValueError("")
        self.points=points
        self.N=len(points[0])
        self.a=1
        self.b=2
        self.c=0.5
        self.d=0.5

    def sort_points(self):
        self.points=sorted(self.points, key=self.f)
        self.evaluate_points()

    def evaluate_points(self):
        values=[]
        for point in self.points:
            values.append(self.f(point))
        self.values=sorted(values)

    def centroidal(self):
        centroid=[0]*self.N
        for i in range(self.N):
            for j in range(len(self.points)-1):
                centroid[i]+=self.points[j][i]
            centroid[i]/=self.N
        self.centroid=centroid

    def reflection(self):
        R=[]
        for i in range(self.N):
            R.append(self.centroid[i]*(1+self.a) - self.a*self.points[-1][i])
        return R
    def extension(self):
        E=[]
        for i in range(self.N):
            E.append(self.centroid[i]*(1+self.b) - self.b*self.points[-1][i])
        return E

    def contraction(self, typ=0):
        C=[]
        c=self.c if typ==0 else -self.c
        for i in range(self.N):
            C.append(self.centroid[i]*(1+c) - c*self.points[-1][i])
        return C

    def shrinking(self):
        for j in range(1,len(self.points)):
            for i in range(self.N):
                self.points[j][i]=self.points[0][i]+ self.d*(self.points[j][i]-self.points[0][i])


    def run(self, steps):
        for step in range(steps):
            self.sort_points()
            if abs(self.values[0]-self.values[-1])<1e-4: break
            self.centroidal()
            R=self.reflection()
            if self.values[0]<=self.f(R)<self.values[-2]:
                self.points[-1]=R
            elif self.f(R)<self.values[0]:
                E=self.extension()
                if self.f(E)<self.f(R):
                    self.points[-1]=E
                else:
                    self.points[-1]=R
            else:
                if self.f(R)<self.values[-1]:
                    C=self.contraction(0)
                    if self.f(C)<self.f(R):
                        self.points[-1]=C
                    else:
                        self.shrinking()
                else:
                    C=self.contraction(1)
                    if self.f(C)<self.f(R):
                        self.points[-1]=C
                    else:
                        self.shrinking()
        return self.points[0]
                


In [142]:
P1=[3,2]
P2=[-3,2]
P3=[0,0]

P=[P1,P2,P3]

simpli=simplex(function,P)
simpli.run(100)

[-1.0030770376324654, 1.007593646645546]

In [122]:
simpli.points[0]

[-3, 2]